In [1]:
%run ../_preamble.ipynb
from _work.src.modules import AutoEncoder, train_model, plot_metrics, plot_tensor
from _work.src.modules.utils import save_metrics, load_metrics
from _work.src.data import small_dataloader as dataloader
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from ray import tune
import ray

/home/jelmar/Github/deep/.venv/lib/python3.9/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


In [4]:
ray.shutdown()

In [5]:
ray.init(local_mode=True, num_cpus=1, num_gpus=1)

2021-07-27 15:11:42,783	INFO services.py:1272 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '172.26.170.179',
 'raylet_ip_address': '172.26.170.179',
 'redis_address': '172.26.170.179:6379',
 'object_store_address': '/tmp/ray/session_2021-07-27_15-11-41_387224_2005/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-07-27_15-11-41_387224_2005/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-07-27_15-11-41_387224_2005',
 'metrics_export_port': 60329,
 'node_id': '1300006a25040a25e3cdf7875a80f2f02e5f782335475577089c51ba'}

In [6]:
def training_function(config):
    model = AutoEncoder(sizes=config['sizes'], batch_norm=config['batch_norm'], dropout=config['dropout'])
    optimizer = torch.optim.AdamW(model.parameters(), lr=config['lr'])
    loss_fn = torch.nn.MSELoss(reduction="sum")

    def callback(epoch, loss, predictions):
        accuracy = predictions["accuracy"].mean()
        tune.report(mean_loss=loss)

    metrics = train_model(
        model,
        optimizer,
        loss_fn,
        dataloader,
        epochs=config['epochs'],
        noise_mean=config['noise_mean'],
        noise_stdev=config['noise_stdev'],
        noise_multiplier=config['noise_multiplier'],
        l1_lambda=config['l1_lambda'],
        callback=callback,
    )

analysis = tune.run(
    training_function,
    config={
        'sizes': [dataloader.dataset.NUM_ATTRIBUTES, 2],
        'batch_norm': tune.grid_search([True, False]),
        'dropout': 0,
        'lr': tune.grid_search([1e-2, 1e-3]),
        'epochs': 200,
        'noise_mean': 0,
        'noise_stdev': 1,
        'noise_multiplier': 0.1,
        'l1_lambda': 0,
    },
    resources_per_trial={'gpu': 1},
)

2021-07-27 15:11:50,116	WARNING tune.py:494 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


training model:   0%|          | 0/200 [00:00<?, ?epochs/s, accuracy=0.444, loss=22.4]

Trial name,status,loc,batch_norm,lr
training_function_34360_00000,RUNNING,,True,0.01
training_function_34360_00001,PENDING,,False,0.01
training_function_34360_00002,PENDING,,True,0.001
training_function_34360_00003,PENDING,,False,0.001


2021-07-27 15:11:50,328	WARNING session.py:51 -- A Tune session already exists in the current process. If you are using ray.init(local_mode=True), you must set ray.init(..., num_cpus=1, num_gpus=1) to limit available concurrency.


2021-07-27 15:14:35,635	WARNING tune.py:506 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 
